<a href="https://colab.research.google.com/github/sugatoray/CodeSnippets/blob/master/Code_Snippets_Map_Dict_List.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # K-Fold validation

# PCA related
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import os, time, json

# progressbar
from tqdm import tqdm, tqdm_notebook, tnrange

%matplotlib inline
%config InlineBackend.figure_format = 'svg' # 'svg', 'retina'
plt.style.use('seaborn-white')

# Map

In [2]:
def pwr(x,n=2):
    return x**n
list(map(lambda x: pwr(x,n=2), [1,2,3,4,5]))

[1, 4, 9, 16, 25]

# Dict

## Make a dict from two lists

In [3]:
keys = np.arange(8)
values = np.arange(1,9)*50
dict(zip(keys, values))

{0: 50, 1: 100, 2: 150, 3: 200, 4: 250, 5: 300, 6: 350, 7: 400}